Корниевская Анастасия

результаты, были получены запуском текущего нотебука на сервисах kaggle

In [0]:
import pickle
import numpy as np
from skimage import io
import PIL
import random

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

import torchvision
from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch

from matplotlib import colors, pyplot as plt
%matplotlib inline

# в sklearn не все гладко, чтобы в colab удобно выводить картинки 
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

для воспроизводимости результатов зафиксируем затравку в рандоме

In [0]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [0]:
TRAIN_DIR = Path('../input/simpsons4/train/')
TEST_DIR = Path('../input/simpsons4/testset/')

#оставим только классы персонажей которых будем распознавать
character_list=['abraham_grampa_simpson','apu_nahasapeemapetilon', 'bart_simpson','charles_montgomery_burns', 'chief_wiggum',\
                'comic_book_guy', 'edna_krabappel', 'homer_simpson', 'kent_brockman', 'krusty_the_clown', 'lenny_leonard',\
                'lisa_simpson', 'marge_simpson', 'mayor_quimby', 'milhouse_van_houten', 'moe_szyslak',
                'ned_flanders', 'nelson_muntz', 'principal_skinner', 'sideshow_bob']
character=[]
for name in character_list:
    character.extend(list(TRAIN_DIR.rglob(name+'/*.jpg')))
train_val_files = sorted(character)
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))

# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")

In [0]:
# в исходный класс была добавлена аугментацияданных для тренировочного набора и немного изменен порядок пребразований
class SimpsonsDataset(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, mode):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files)
     
        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)

            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)
                      
    def __len__(self):
        return self.len_
      
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
  
    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        transform=[]
        if self.mode == 'train':
            transform = transforms.Compose([
            transforms.Resize([RESCALE_SIZE, RESCALE_SIZE]),
            #аугментация
            transforms.RandomRotation(20),
            transforms.RandomHorizontalFlip(p=0.5),  
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        else:
            transform = transforms.Compose([
            transforms.Resize([RESCALE_SIZE, RESCALE_SIZE]),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        x = self.load_sample(self.files[index])
        #x = np.array(x / 255, dtype='float32')
        x = transform(x)
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y
        

In [0]:
#ничего не менялось
def fit_epoch(model, train_loader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
  
    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
              
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc

In [0]:
#ничего не менялось
def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    return val_loss, val_acc

In [0]:
#был добавлен планировик для изменения lr и сохранение весов, но на них в каггле не хватает памяти
def train(train_files, val_files, model, epochs, batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = torch.optim.Adam(model.parameters(), lr=0.0001)
        criterion = nn.CrossEntropyLoss()
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min')
        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt)
            print("loss", train_loss)
            
            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
            scheduler.step(val_loss)
            #torch.save(model.state_dict(), str(model)+str(epoch)+'.pth')
            
    return history

In [0]:
#ничего не менялось
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [0]:
from sklearn.model_selection import train_test_split

train_val_labels = [path.parent.name for path in train_val_files]
train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels, random_state=42,shuffle=True)

импортируем и переопределуем последние слои у МНОГАМОДЕЛЕЙ чтобы строить ансамбль.

In [0]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet=models.densenet161(pretrained=True)
googlenet = models.googlenet(pretrained=True)
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
mobilenet = models.mobilenet_v2(pretrained=True)
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
mnasnet = models.mnasnet1_0(pretrained=True)

In [0]:
n_classes = len(np.unique(train_val_labels))

resnet18.fc = nn.Linear(resnet18.fc.in_features, n_classes)
alexnet.classifier= nn.Sequential(
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=9216, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=4096, out_features=n_classes, bias=True)
        )       
squeezenet.classifier=nn.Sequential(
            nn.Dropout(p=0.5, inplace=False),
            nn.Conv2d(512, n_classes, kernel_size=(1, 1), stride=(1, 1)),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(output_size=(1, 1))
        )
vgg16.classifier =nn.Sequential(
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=n_classes, bias=True)
        )
densenet.classifier=nn.Linear(in_features=2208, out_features=n_classes, bias=True)
googlenet.fc=nn.Linear(in_features=1024, out_features=n_classes, bias=True)
shufflenet.fc=nn.Linear(in_features=1024, out_features=n_classes, bias=True)
mobilenet.classifier=nn.Sequential(
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(in_features=1280, out_features=n_classes, bias=True)
        )
resnext50_32x4d.fc=nn.Linear(in_features=2048, out_features=n_classes, bias=True)
wide_resnet50_2.fc=nn.Linear(in_features=2048, out_features=n_classes, bias=True)
mnasnet.classifier=nn.Sequential(
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(in_features=1280, out_features=n_classes, bias=True)
  )

In [0]:
resnet18.to(DEVICE)
alexnet.to(DEVICE)
squeezenet.to(DEVICE)
vgg16.to(DEVICE)
densenet.to(DEVICE)
googlenet.to(DEVICE)
shufflenet.to(DEVICE)
mobilenet.to(DEVICE)
resnext50_32x4d.to(DEVICE)
wide_resnet50_2.to(DEVICE)
mnasnet.to(DEVICE)

In [0]:
val_dataset = SimpsonsDataset(val_files, mode='val')    
train_dataset = SimpsonsDataset(train_files, mode='train')

In [0]:
train(train_dataset, val_dataset, model=resnet18, epochs=10, batch_size=32)
train(train_dataset, val_dataset, model=alexnet, epochs=10, batch_size=32)
train(train_dataset, val_dataset, model=squeezenet, epochs=10, batch_size=32)
train(train_dataset, val_dataset, model=vgg16, epochs=10, batch_size=32)
train(train_dataset, val_dataset, model=densenet, epochs=10, batch_size=32)
train(train_dataset, val_dataset, model=googlenet, epochs=10, batch_size=32)
train(train_dataset, val_dataset, model=shufflenet, epochs=10, batch_size=32)
train(train_dataset, val_dataset, model=mobilenet, epochs=10, batch_size=32)
train(train_dataset, val_dataset, model=resnext50_32x4d, epochs=10, batch_size=32)
train(train_dataset, val_dataset, model=wide_resnet50_2, epochs=10, batch_size=32)
train(train_dataset, val_dataset, model=mnasnet, epochs=10, batch_size=32)

делаем предсказания

In [0]:
test_dataset = SimpsonsDataset(test_files, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)


resnet18probs = predict(resnet18, test_loader)
alexnetprobs = predict(alexnet, test_loader)
squeezenetprobs = predict(squeezenet, test_loader)
vgg16probs = predict(vgg16, test_loader)
densenetprobs = predict(densenet, test_loader)
googlenetprobs = predict(googlenet, test_loader)
shufflenetprobs = predict(shufflenet, test_loader)
mobilenetprobs = predict(mobilenet, test_loader)
resnext50_32x4dprobs = predict(resnext50_32x4d, test_loader)
wide_resnet50_2probs = predict(wide_resnet50_2, test_loader)
mnasnetprobs = predict(mnasnet, test_loader)

probs1=resnet18probs + alexnetprobs + squeezenetprobs + vgg16probs + densenetprobs +  googlenetprobs +\
        shufflenetprobs + mobilenetprobs + resnext50_32x4dprobs + wide_resnet50_2probs + mnasnetprobs
probs2=np.maximum.reduce([resnet18probs , alexnetprobs , squeezenetprobs , vgg16probs , densenetprobs , googlenetprobs ,\
        shufflenetprobs , mobilenetprobs , resnext50_32x4dprobs , wide_resnet50_2probs , mnasnetprobs])
from scipy import stats
probs3=stats.mode([np.argmax(resnet18probs, axis=1), np.argmax(alexnetprobs, axis=1), np.argmax(squeezenetprobs, axis=1), \
        np.argmax(vgg16probs, axis=1), np.argmax(densenetprobs, axis=1), \
        np.argmax(googlenetprobs, axis=1), np.argmax(shufflenetprobs, axis=1), np.argmax(mobilenetprobs, axis=1), \
        np.argmax(resnext50_32x4dprobs, axis=1), np.argmax(wide_resnet50_2probs, axis=1), np.argmax(mnasnetprobs, axis=1)])[0]

label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

preds1 = label_encoder.inverse_transform(np.argmax(probs1, axis=1))
preds2 = label_encoder.inverse_transform(np.argmax(probs2, axis=1))
preds3 = label_encoder.inverse_transform(probs3[0])

test_filenames = [path.name for path in test_dataset.files]


In [0]:
import pandas as pd
#my_submit = pd.read_csv("gdrive/My Drive/simpsons/data/labels.csv")
my_submit1 = pd.DataFrame({'Id': test_filenames, 'Expected': preds1})
my_submit2 = pd.DataFrame({'Id': test_filenames, 'Expected': preds2})
my_submit3 = pd.DataFrame({'Id': test_filenames, 'Expected': preds3})


Все сохраняем

In [0]:
my_submit1.to_csv('ansumble1.csv', index=False)
my_submit2.to_csv('ansumble2.csv', index=False)
my_submit3.to_csv('ansumble3.csv', index=False)

In [0]:
pd.DataFrame(resnet18probs).to_csv('resnet18probs.csv', index=False)
pd.DataFrame(alexnetprobs).to_csv('alexnetprobs.csv', index=False)
pd.DataFrame(squeezenetprobs).to_csv('squeezenetprobs.csv', index=False)
pd.DataFrame(vgg16probs).to_csv('vgg16probs.csv', index=False)
pd.DataFrame(densenetprobs).to_csv('densenetprobs.csv', index=False)
pd.DataFrame(googlenetprobs).to_csv('googlenetprobs.csv', index=False)
pd.DataFrame(shufflenetprobs).to_csv('shufflenetprobs.csv', index=False)
pd.DataFrame(mobilenetprobs).to_csv('mobilenetprobs.csv', index=False)
pd.DataFrame(resnext50_32x4dprobs).to_csv('resnext50_32x4dprobs.csv', index=False)
pd.DataFrame(wide_resnet50_2probs).to_csv('wide_resnet50_2probs.csv', index=False)
pd.DataFrame(mnasnetprobs).to_csv('mnasnetprobs.csv', index=False)

Что было вообще

обучанеи своей нейронной сети
Public Score: 0.96632

дообучение моделей с заморозкой слоев
Public Score: 0.76094

дообучение моделеи ResNet18 с без заморозки слоев
Public Score: 0.98653


ансамбль с усреднением веоятности (my_submit1)
Public Score: 0.99663


# и вишенка на торте - Public Score: 1.0

в процессе обучения модели  давали разный скор на трейне и валидации. после отправки трех сабмитов из данного нотебука было выясненно, что в лучшем случае не корректно распознается одна картинка на паблике. Также были вручную отсмотрены случаи, где по результатам голосования (сабмит3) решение принято меньше,чем половиной голосов. В итоге было отобрано и в ручную определен класс для двух картинок. На основании внутренних суждений, скора при обучении и ранее полученных резульатах отправки сабмита была сделана следуюая финальная модель 

In [0]:
probs=resnet18probs + 0.5*alexnetprobs + squeezenetprobs + 5*vgg16probs + 3*densenetprobs +  4*googlenetprobs +\
        shufflenetprobs + mobilenetprobs + 0.5*resnext50_32x4dprobs + wide_resnet50_2probs + mnasnetprobs
preds = label_encoder.inverse_transform(np.argmax(probs.values, axis=1))
my_submit = pd.DataFrame({'Id': test_filenames, 'Expected': preds})
my_submit.to_csv('total.csv', index=False)